# Модуль 2.3 — Fallbacks и Streaming

**Цель:** сделать цепочки устойчивыми к ошибкам и научиться выводить ответы потоком.

**Что сделаем:**
- установим библиотеки
- настроим ключ и base URL
- добавим `with_fallbacks()`
- покажем потоковую генерацию

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Fallbacks: отказоустойчивость

Если основная модель временно недоступна или возвращает ошибку, мы автоматически используем запасную.
Это критично для продакшена: пользователь получает ответ, даже если что-то сломалось.

In [ ]:
from langchain_openai import ChatOpenAI

base_url = os.environ.get("OPENAI_BASE_URL")

primary = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

fallback = ChatOpenAI(
    model="gpt-5-mini",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

robust_chain = primary.with_fallbacks([fallback])

response = robust_chain.invoke("Объясни, что такое LangChain в 3 предложениях.")
print(response.content)

## Streaming: потоковая генерация

Потоковый вывод полезен для интерфейсов — пользователь видит ответ по мере генерации.

Мы используем `stream()` и выводим токены в реальном времени.

In [ ]:
streaming_model = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.5,
    max_tokens=256,
    base_url=base_url,
    streaming=True,
)

print("Ответ (стриминг):")
for chunk in streaming_model.stream("Дай краткий совет по изучению LangChain."):
    if chunk.content:
        print(chunk.content, end="", flush=True)
print()

## Практические рекомендации

- Для fallback выбирайте модель дешевле и быстрее, чтобы снижать затраты.
- Логируйте, когда срабатывает fallback — это сигнал качества/доступности.
- Streaming лучше включать в UI, но в ноутбуке тоже полезно для демонстрации.
- Если нужна строгая структура — streaming не мешает, но требует парсинга после полного ответа.